# Tratamento de dados - Secretaria de segurança do estado do Paraná

## Descrição do notebook

O objetivo desse notebook é realizar o tratamento dos dados fornecidos pela Secretaria de Segurança Pública do Estado do Paraná para que possa ser analisado juntamente com os dados do estado paraens.

Nessa versão, são utilizados quatro arquivos do tipo CSV e para manipulação de dados é utilizado o pacote pandas.

**Observação:** O código contido nesse notebook foi baseado na solução desenvolvida por Mariana Kniss (https://www.linkedin.com/in/mariana-kniss-471ba0196/), para validar o conteúdo de seu trabalho de conclusão de curso, intitulado "ANÁLISE DE DADOS CRIMINAIS SOBRE VIOLÊNCIA CONTRA A MULHER NOS ESTADOS PARÁ E PARANÁ".

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
4. [Agrupamento dos dados](#agrupamento-dos-dados)
5. [Visão geral do conjunto de dados](#visao-geral-do-conjunto-de-dados)
6. [Filtros, tratamentos e padronização de valores](#filtros-tratamentos-e-padronizacao-de-valores)
7. [Ordenação das colunas](#ordernacao-das-colunas)
8. [Exportação dos dados formatados](#exportacao-dos-dados-formatados)

----


## Importação dos módulos e pacotes necessários <a id="importacao-dos-modulos-e-pacotes-necessarios"></a>

In [ ]:
# Pandas (https://pandas.pydata.org/)
import pandas as pd

# Datetime (https://docs.python.org/3/library/datetime.html)
import datetime as dt

# Funções de internacionalização para formatar datas, número etc. (https://docs.python.org/3/library/locale.html)
import locale

## Carregamento dos dados

In [ ]:
# Dados sobre as ocorrências
df_ocorrencias = pd.read_csv('./dados/PR/Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv', sep = ';')

# Dados sobre as naturezas
df_naturezas = pd.read_csv('./dados/PR/Naturezas das Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv', sep = ';')

# Dados sobre as vítimas
df_vitimas = pd.read_csv('./dados/PR/Vitimas das Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv', sep = ';')

# Dados sobre os autores
df_autores = pd.read_csv('./dados/PR/Autores das Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv', sep = ';')

### Boletins de ocorrências

In [ ]:
df_ocorrencias.head(10)

In [ ]:
df_ocorrencias.columns

In [ ]:
# Verificando o conteúdo da coluna 'identificacao_fato'
df_ocorrencias.identificacao_fato.value_counts()

In [ ]:
# Quantidade de registros encontrados no arquivo de ocorrências
len(df_ocorrencias)

In [ ]:
# Quantidade de valores únicos para identificação de boletins de ocorrências
df_ocorrencias['id_boletim'].nunique()

In [ ]:
df_ocorrencias_ate_2022 = df_ocorrencias[df_ocorrencias['ano_fato'] <= 2022]
len(df_ocorrencias_ate_2022)

In [ ]:
df_ocorrencias_ate_2022['identificacao_fato'].value_counts()

Como pode ser verificado, cada linha do arquivo de ocorrências representa os dados de um BO.

- **Número de registros no arquivo:** 340.478
- **Número de boletins de ocorrências:** 340.478
- **Número de ocorrências entre 2018 e 2022:** 290.033 => Todos identificados como 'Violencia_Domestica_Contra_Mulher'

-----

### Vítimas

In [ ]:
df_vitimas.head(10)

In [ ]:
len(df_vitimas)

Enquanto há 340.478 ocorrências, há 350.240 vítimas, representando uma diferença de 9.762. Ou seja, há casos de registros de violências com mais de uma vítima. Verificar.

In [ ]:
df_vitimas_ate_2022 = df_vitimas[df_vitimas['ano_fato'] <= 2022]
len(df_vitimas_ate_2022)

#### Remoção das colunas já existentes no conjunto das ocorrências

Esse procedimento foi adotado como precendente à fusão dos conjuntos de dados, visto que colunas de mesmo nome são renomeadas no método merge().

In [ ]:
# Cria uma lista com as colunas das ocorrências
colunas_ocorrencias = df_ocorrencias.columns
# Cria uma lista com as colunas das vítimas
colunas_vitimas = df_vitimas.columns
# Cria uma lista com as colunas em comum das duas listas anteriores
colunas_comuns = set(colunas_ocorrencias) & set(colunas_vitimas)
# Remove dessa lista em comum a coluna 'id_boletim' que deverá ser mantida nos dois conjuntos de dados para servir como chave
colunas_comuns.remove('id_boletim')
# Cria um novo dataframe das vítimas, sem as colunas que se repetem no conjunto das ocorrências
df_vitimas_sem_colunas_comuns = df_vitimas.drop(columns=colunas_comuns)
df_vitimas_sem_colunas_comuns.columns

#### Renomeando as colunas

Algumas colunas possuem o mesmo nome para o conjunto de vítimas e de autores. Assim, foram adicionados prefixos para diferenciar esses dados. 

Além disso, algumas variáveis foram renomeadas para manter o mesmo padrão utilizado nos dados do estado do Pará, como 'raca_cor' alterada para '_cor_pele'.

In [ ]:
df_vitimas_sem_colunas_comuns.rename(columns={
	'faixa_etaria'		: 'vit_faixa_etaria',
	'sexo'				: 'vit_sexo',
	'raca_cor'			: 'vit_cor_pele',
	'grau_instrucao'	: 'vit_grau_inst',
	'profissao'			: 'vit_profissao'}, inplace=True)

df_vitimas_sem_colunas_comuns.columns

----

### Natureza

In [ ]:
df_naturezas.head(10)

#### Remoção das colunas já existentes no conjunto das ocorrências

In [ ]:
colunas_naturezas = df_naturezas.columns
colunas_comuns = set(colunas_ocorrencias) & set(colunas_naturezas)
colunas_comuns.remove('id_boletim')
df_naturezas_sem_colunas_comuns = df_naturezas.drop(columns=colunas_comuns)

df_naturezas_sem_colunas_comuns.columns

----

### Autores

In [ ]:
df_autores.head(10)

#### Remoção das colunas já existentes no conjunto das ocorrências

In [ ]:
colunas_autores = df_autores.columns
colunas_comuns = set(colunas_ocorrencias) & set(colunas_autores)
colunas_comuns.remove('id_boletim')
df_autores_sem_colunas_comuns = df_autores.drop(columns=colunas_comuns)

df_autores_sem_colunas_comuns.columns

#### Renomeando as colunas

In [ ]:
df_autores_sem_colunas_comuns.rename(columns={
	'faixa_etaria'		: 'aut_faixa_etaria',
	'sexo'				: 'aut_sexo',
	'raca_cor'			: 'aut_cor_pele',
	'grau_instrucao'	: 'aut_grau_inst',
    'parentesco'		: 'grau_de_relacionamento',
	'profissao'			: 'aut_profissao'}, inplace=True)

df_autores_sem_colunas_comuns.columns

## Agrupamento dos dados

Os procedimentos nesta seção se destinam a agrupamento os dados dos quatro arquivos distintos relacionados à violência contra a mulher.

O campo utilizado como 'chave primária' é a identificação do boletim de ocorrência (*id_boletim*).

Assim, é importante considerar esse procedimento para as análises estatísticas, visto que uma mesma ocorrência pode apresentar mais de um registros no conjunto de dados final.

In [ ]:
# Fusão dos dataframes carregados anteriormente
df_ocorrencias_vitimas = pd.merge(df_ocorrencias, df_vitimas_sem_colunas_comuns, on='id_boletim', how='left')
df_ocorrencias_vitimas_naturezas = pd.merge(df_ocorrencias_vitimas, df_naturezas_sem_colunas_comuns, on='id_boletim', how='left')
df_completo = pd.merge(df_ocorrencias_vitimas_naturezas, df_autores_sem_colunas_comuns, on='id_boletim', how='left')

In [ ]:
df_completo.columns

In [ ]:
len(df_completo)

## Visão geral do conjunto de dados <a id="visao-geral-do-conjunto-de-dados"></a>

In [ ]:
# Contagem do número de variáveis
v = len(df_completo.columns)
# Contagem do número de registros
n = len(df_completo)

print(f'O conjunto de dados possui {n} registros e {v} variáveis.')

In [ ]:
df_completo.head()

In [ ]:
# Descrição dos tipos dos dados
df_completo.info()

----

## Filtros, tratamentos e padronização de valores <a id="filtros-tratamentos-e-padronizacao-de-valores"></a>

### Filtros

#### Classe

É estranho que a identificação de todos os registro consta como 'Violencia_Domestica_Contra_Mulher'. Mesmo assim, será mantido o filtro por palavras-chave?

In [ ]:
df_completo['identificacao_fato'].value_counts()

**Visualizando os valores para a variável 'classe_motivo'.**

In [ ]:
# Cria um novo dataframe para guardar os dados dos registros por ano
df_registros = df_completo['classe_motivo'].value_counts().reset_index().rename(columns={'index': 'Index', 'classe_motivo': 'Classe motivo', 'count': 'Total'}).sort_values(by='Total', ascending=False).reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores anuais
df_registros['Porcentagem'] = df_registros.Total / df_registros.Total.sum()
# Exibe a tabela
df_registros.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

Definição de palavras-chave para filtrar apenas os casos que são específicos de violência contra a mulher.

In [ ]:
palavras_chave = ["mulher", "sexual", "medidas protetivas", "13772"]

In [ ]:
df_VCM = df_completo[(df_completo['classe_motivo'].notnull()) & (df_completo['classe_motivo'].str.lower().str.contains('|'.join(palavras_chave)))]
len(df_VCM)

Filtrando apenas casos ocorridos até 2022.

In [ ]:
df_VCM = df_VCM[df_VCM['ano_fato'] <= 2022]
len(df_VCM)

### Investigando os registros em relação aos boletins de ocorrência

Identificando a quantidade de boletins de ocorrência únicos.

In [ ]:
len(df_VCM.drop_duplicates(subset='id_boletim'))

Verificando os boletins com mais de um envolvido.

In [ ]:
# Identifica os valores distintos de identificação de boletins de ocorrência
valores_unicos_bo = df_VCM['id_boletim'].unique()
# Contagem do número de ocorrências para cada valor único identificado
contagem_valores_unicos_bo = df_VCM['id_boletim'].value_counts()
# Identifica apenas os boletins que apresentam mais de um registro, isto é, ou possuem mais de uma vítima, mais de um autor e/ou mais de uma natureza (class_motivo)
boletins_com_mais_de_um_registro = contagem_valores_unicos_bo[contagem_valores_unicos_bo > 1]
print(boletins_com_mais_de_um_registro)

In [ ]:
# Exibe os dados do boletim '20221093794', com seis registros
df_boletim = df_VCM[df_VCM['id_boletim'] == 2022646186]
df_boletim.to_csv('./dados-tratados/boletim-unico.csv')

Verificando as vítimas que aparecem mais de uma vez nos registros.

In [ ]:
# Identifica os valores distintos de identificação de boletins de ocorrência
valores_unicos_vitima = df_VCM['vitima'].unique()
# Contagem do número de ocorrências para cada valor único identificado
contagem_valores_unicos_vitima = df_VCM['vitima'].value_counts()
# Identifica apenas os boletins que apresentam mais de um registro, isto é, ou possuem mais de uma vítima, mais de um autor e/ou mais de uma natureza (class_motivo)
boletins_com_mais_de_uma_vitima = contagem_valores_unicos_vitima[contagem_valores_unicos_vitima > 1]
print(boletins_com_mais_de_uma_vitima)

In [ ]:
# Exibe os dados da vítima 'id_env_vitima20229092241', com quatro registros
df_vitima = df_VCM[df_VCM['vitima'] == 'id_env_vitima20229092241']
df_vitima.to_csv('./dados-tratados/vitima-unica.csv')

### Tratamentos


In [ ]:
print(df_VCM.columns)

#### Removação de atributos não relevantes para as análises

In [ ]:
# Remoção de colunas desnecessárias
colunas_para_remover = ['fx_4_hr', # Padrão de faixa horária em quatro horas
						'fx_12_hr', # Padrão de faixa horária em doze horas
						'hora_registro', # Horário de registro
						'hora_fato', # Horário do fato
						'causa_presumivel', # Causa presumível
						'distrito', # Distrito
						'regional', # Regional
						#'id_boletim', # Identificação do BO --> Será mantido
						#'vitima', # Identificação da vítima --> Será mantido
						#'autor', # Identificação do autor --> Será mantido
						'identificacao_fato', # Identificação dos fatos (Todos estão com o valor 'Violencia_Domestica_Contra_Mulher')
						]

df_VCM = df_VCM.drop(columns=colunas_para_remover)

In [ ]:
print(df_VCM.columns)

#### Dias da semana

Converte os valores para letras maiúsculas.

In [ ]:
df_VCM['dia_semana'] = df_VCM['dia_semana'].str.upper()

#### Faixa horária

Separando as colunas referentes às datas e horários do fato e do registro em duas colunas distintas, uma para o horário e outra para a data.

In [ ]:
if not 'hora_registro' in df_VCM.columns:
	df_VCM['hora_registro'] = pd.to_datetime(df_VCM['data_registro'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time

df_VCM['data_registro'] = pd.to_datetime(df_VCM['data_registro'].astype('str').str.slice(0, 10))

if not 'hora_fato' in df_VCM.columns:
	df_VCM['hora_fato'] = pd.to_datetime(df_VCM['data_fato'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time

df_VCM['data_fato'] = pd.to_datetime(df_VCM['data_fato'].astype('str').str.slice(0, 10))

In [ ]:
for h in df_VCM['hora_fato']:
	print(h)
	break

In [ ]:

# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 6, 12, 18, 24]
labels = ['Madrugada', 'Manhã', 'Tarde', 'Noite']
categories = labels + ['Outro']

# Cria uma nova coluna com a hora do fato no formato inteiro
df_VCM['hora_fato_int'] = df_VCM['hora_fato'].apply(lambda x: x.hour)

# Período que ocorreu o fato (Manhã, Tarde, Noite ou Madrugada)
df_VCM['faixa_horaria'] = pd.cut(df_VCM['hora_fato_int'], bins=intervals, labels=labels, right=False)
df_VCM['faixa_horaria'] = pd.Categorical(df_VCM['faixa_horaria'], categories=categories)
df_VCM['faixa_horaria'] = df_VCM['faixa_horaria'].fillna('Outro')

# Período que ocorreu o fato (['00h - 06h', '06h - 12h', '12h - 18h' ou '18h - 00h')
labels = ['00h - 06h', '06h - 12h', '12h - 18h', '18h - 00h']
categories = labels + ['Outro']

df_VCM['faixa_horaria_6h'] = pd.cut(df_VCM['hora_fato_int'], bins=intervals, labels=labels, right=False)
df_VCM['faixa_horaria_6h'] = pd.Categorical(df_VCM['faixa_horaria_6h'], categories=categories)
df_VCM['faixa_horaria_6h'] = df_VCM['faixa_horaria_6h'].fillna('Outro')

intervals = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
labels = ['00h - 02h', '02h - 04h', '04h - 06h', '06h - 08h', '08h - 10h', '10h - 12h', '12h - 14h', '14h - 16h', '16h - 18h', '18h - 20h', '20h - 22h', '22h - 00h']
categories = labels + ['Outro']

df_VCM['faixa_horaria_2h'] = pd.cut(df_VCM['hora_fato_int'], bins=intervals, labels=labels, right=False)
df_VCM['faixa_horaria_2h'] = pd.Categorical(df_VCM['faixa_horaria_2h'], categories=categories)
df_VCM['faixa_horaria_2h'] = df_VCM['faixa_horaria_2h'].fillna('Outro')

# Exibindo a coluna com o horário e as demais colunas criadas
df_VCM[['hora_fato', 'hora_fato_int', 'faixa_horaria', 'faixa_horaria_6h', 'faixa_horaria_2h']].head(10)

In [ ]:
df_VCM['vit_dt_nasc']

Removendo o horário das datas de nascimento das vítimas e dos autores.

In [ ]:
df_VCM['vit_dt_nasc'] = pd.to_datetime(df_VCM['vit_dt_nasc'].astype('str').str.slice(0, 10), errors='coerce')
df_VCM['aut_dt_nasc'] = pd.to_datetime(df_VCM['aut_dt_nasc'].astype('str').str.slice(0, 10), errors='coerce')

## Ordenação das colunas <a id="ordernacao-das-colunas"></a>

### Definindo a nova ordem das colunas

Cria uma nova coluna para especificar o estado do Paraná.

In [ ]:
df_VCM['estado'] = 'PR'

### Definindo a nova ordem das colunas

In [ ]:
df_VCM = df_VCM[['data_registro', 'hora_registro', 'data_fato', 'hora_fato', 'dia_semana', 'faixa_horaria_2h', 'faixa_horaria_6h', 'classe_motivo', 'mes_fato', 'ano_registro', 'ano_fato', 'registros', 'municipio', 'bairro', 'local_ocorrencia', 'meio_empregado', 'mes_registro', 'vit_dt_nasc', 'vit_idade', 'vit_faixa_etaria', 'vit_cor_pele', 'vit_grau_inst', 'vit_profissao', 'aut_dt_nasc', 'aut_idade', 'aut_faixa_etaria', 'aut_cor_pele', 'aut_grau_inst', 'aut_profissao', 'grau_de_relacionamento', 'faixa_horaria', 'estado', 'id_boletim', 'vitima', 'autor']]

## Exportação dos dados formatados <a id="exportacao-dos-dados-formatados"></a>

In [ ]:
df_VCM.to_csv('./dados-tratados/' + dt.datetime.now().strftime('%Y%m%d%H%M%S') + '-dados_VCM_PR-pandas.csv', index=False, sep='|', encoding='utf-8')